In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import re
import string

import statsmodels.api as sm
from sklearn.metrics import log_loss

In [3]:
df_fake = pd.read_csv("fake.csv")
df_true = pd.read_csv("true.csv")

In [4]:
df_fake.tail(10)

,title,content,date,sub-category,label
6050,"Divonis 3 Bulan, Kasus Pedangdut Xena Xenita y...","Akibatnya, , yang dijerat pasal perzinaan, div...",19-Sep-19,Seleb,1
6051,"Ramalan Zodiak Cinta Kamis 19 September 2019, ...",- Bagaimana kelangsungan kisah asmara konon bi...,19-Sep-19,Lifestyle,1
6052,"Namanya Disebut Jadi Calon Wali Kota Manado, E...",Nama gitaris grup band Ungu Franco Wellyjat a...,19-Sep-19,Seleb,1
6053,Catatan Real Madrid Kalah dari PSG di Liga Cha...,"- , berhasil menaklukan hadangan juara , tiga...",19-Sep-19,Super Skor,1
6054,"Ramalan Zodiak Kamis 19 September 2019, Scorpi...","- Berikut ramalan lengkap 12 zodiak, Kamis (19...",19-Sep-19,Lifestyle,1
6055,Hari Terakhir Lowongan Kerja Bank Indonesia Se...,"Hari terakhir pendaftaran lowongan kerja , ,,...",19-Sep-19,Nasional,1
6056,"Kronologi Duel Pelajar SMP, Korban Meninggal D...","Kasatreskrim Polres,Grobogan,AKP Agus Supriyad...",19-Sep-19,Regional,1
6057,10 Wisata Negeri di Atas Awan yang Cocok Dikun...,¬†Tengah viral di medsos tentang beberapa temp...,19-Sep-19,Travel,1
6058,Ekspresi Neymar dan Mbappe saat PSG Tekuk Real...,"- , mengawali langkah yang baik menuju gelar ...",19-Sep-19,Super Skor,1
6059,"Presenter, Aktor, Produser dan Pebisnis, Profe...",", kini tak hanya dikenal sebagai , saja. Ia j...",19-Sep-19,Seleb,1


In [5]:
df_true.head(10)

,title,content,date,sub-category,label
0,"Masuk Radar Pilwalkot Medan, Menantu Jokowi Be...","Medan - Menantu Jokowi, Bobby Afif Nasution, m...",10-Sep-19,Berita,0
1,Malaysia Sudutkan RI: Isu Kabut Asap hingga In...,Jakarta - Malaysia kembali menyerang pemerinta...,10-Sep-19,Berita,0
2,"Kemensos Salurkan Rp 7,3 M bagi Korban Kerusuh...",Jakarta - Kementerian Sosial (Kemensos) menyal...,10-Sep-19,Berita,0
3,"Uang Rp 1,6 M yang Raib di Parkiran Kantor Gub...",Medan - Uang sebanyak Rp 1.672.985.500 milik P...,9/10/2019,Berita,0
4,MPR: Amandemen UUD 1945 Tak Akan Melebar ke Ma...,Jakarta - Anggota Badan Pengkajian MPR RI Bamb...,10-Sep-19,Berita,0
5,"Peringati Tahun Baru Islam, Banyuwangi Kembali...",Banyuwangi - Memperingati Tahun Baru Islam 144...,10-Sep-19,Berita-jawa-timur,0
6,Wanita Asal Kendari Nyaris Diperkosa di Jalana...,Makassar - Perempuan berusia 20 tahun asal Kot...,10-Sep-19,Berita,0
7,HNW Sebut GBHN Jamin Rencana Pembangunan RI Ja...,Jakarta - Wakil Ketua MPR RI Hidayat Nur Wahid...,10-Sep-19,Berita,0
8,JakPro Jawab Protes Adhi Karya soal Tender Pro...,Jakarta - PT Jakarta Propertindo (JakPro) menj...,10-Sep-19,Berita,0
9,"Mayat di Ladang Tebu Malang Adalah Begal, Dibu...","Malang - Dalam waktu kurang dari 24 jam, polis...",10-Sep-19,Berita-jawa-timur,0


In [6]:
df_fake["class"] = 0
df_true["class"] = 1

In [7]:
df_fake.shape, df_true.shape

((6060, 6), (8105, 6))

In [8]:
df_fake_manual_testing = df_fake.tail(10)
for i in range(6059, 6049, -1):
    df_fake.drop([i], axis = 0, inplace = True)
    
df_true_manual_testing = df_true.tail(10)
for i in range(8104, 8094, -1):
    df_true.drop([i], axis = 0, inplace = True)

In [9]:
df_manual_testing = pd.concat([df_fake_manual_testing, df_true_manual_testing], axis = 0)
df_manual_testing.to_csv("manual_testing.csv")

In [10]:
df_merge = pd.concat([df_fake, df_true], axis=0)
df_merge.head(10)

,title,content,date,sub-category,label,class
0,Viral! Driver Ojol di Bekasi Antar Pesanan Mak...,Bekasi - Seorang driver ojek online di Bekasi ...,10-Sep-19,Berita,1,0
1,"Terkait Mayat Bayi Mengenaskan di Tangerang, S...",Tangerang - Polisi menangkap pria berinisial J...,10-Sep-19,Berita,1,0
2,Muncul Istana di Tengah Rekonsiliasi Rusuh Papua,Jakarta - Bumi Cenderawasih telah memulihkan d...,10-Sep-19,Berita,1,0
3,"Ada Motor Nyangkut di Atas Bambu di Sleman, Ko...",Sleman - Sepeda motor warna putih berpelat AB ...,10-Sep-19,Berita-jawa-tengah,1,0
4,Pesan Gamblang Poyuono Menolak Revisi UU KPK,Jakarta - Pesan lantang menolak revisi Undang-...,10-Sep-19,Berita,1,0
5,Kocak! Maling di Rumah Mewah Jakut Terekam CCT...,Jakarta - Sebuah rekaman CCTV viral di media s...,10-Sep-19,Berita,1,0
6,"Hampir Seluas Jabar, Ini Tantangan Sintang Sia...",Sintang - Akses pendidikan dasar di daerah ped...,10-Sep-19,Berita,1,0
7,"Viral Video Diduga Baku Tembak di Sleman, Ini ...",Sleman - Sebuah video yang menyampaikan ketera...,10-Sep-19,Berita-jawa-tengah,1,0
8,Giliran Alumni Unnes Serukan Tolak Revisi UU KPK,Semarang - Gelombang penolakan terhadap revisi...,10-Sep-19,Berita-jawa-tengah,1,0
9,"Waspada! Ada Penipuan Catut Pertamina, Korbann...",Semarang - Penipuan dengan modus mengatasnamak...,10-Sep-19,Berita-jawa-tengah,1,0


In [11]:
df = df_merge.drop(["title", "sub-category", "date", "label"], axis = 1)
df.head(10)

,content,class
0,Bekasi - Seorang driver ojek online di Bekasi ...,0
1,Tangerang - Polisi menangkap pria berinisial J...,0
2,Jakarta - Bumi Cenderawasih telah memulihkan d...,0
3,Sleman - Sepeda motor warna putih berpelat AB ...,0
4,Jakarta - Pesan lantang menolak revisi Undang-...,0
5,Jakarta - Sebuah rekaman CCTV viral di media s...,0
6,Sintang - Akses pendidikan dasar di daerah ped...,0
7,Sleman - Sebuah video yang menyampaikan ketera...,0
8,Semarang - Gelombang penolakan terhadap revisi...,0
9,Semarang - Penipuan dengan modus mengatasnamak...,0


In [12]:
df = df.sample(frac=1)

In [13]:
df.head(10)

,content,class
1101,", , - Ketua Harian¬†",1
2202,Beberapa hari terakhir aksi unjuk rasa atau d...,0
7349,Nama perwira tinggi dari Angkatan Udara (AU)...,1
3887,Orang yang mampu memahami emosi orang lain me...,0
1021,"- Terpidana kerusuhan 22 Mei, Rendy Bugis Pet...",0
2528,", - Dinas Pariwisata Pemerintah Kabupaten Lam...",1
4280,"IHRAM.CO.ID, JAKARTA -- Salah satu tokoh bangs...",1
6019,"-,dan Faisal Harris sempat viral beberapa wakt...",1
7468,Penyesalan selalu datang di akhir. Kalimat t...,1
1695,"- Prediksi Timnas , dalam , zona Asia dapat d...",0


In [14]:
df.isnull().sum()

content    18
class       0
dtype: int64

In [15]:
def word_drop(content):
    content = content.lower()
    content = re.sub('\[.*?\]', '', content)
    content = re.sub("\\W", " ", content)
    content = re.sub('https?://\S+|www\.\S+', '', content)
    content = re.sub('Äìôóò', '', content)
    content = re.sub('¬†', '', content)
    content = re.sub('<.*?>+', '', content)
    content = re.sub('[%s]' % re.escape(string.punctuation), '', content)
    content = re.sub('\n', '', content)
    content = re.sub('\w*\d\w*', '', content)
    return content

In [16]:
df["content"] = df['content'].astype(str)
df["content"].apply(word_drop)

1101                                       ketua harian  
2202     beberapa hari terakhir aksi unjuk rasa atau d...
7349      nama perwira tinggi dari angkatan udara  au ...
3887     orang yang mampu memahami emosi orang lain me...
1021       terpidana kerusuhan  mei  rendy bugis petta...
                              ...                        
7499       persatuan sepak bola seluruh indonesia     ...
1566    tempo co  jakarta   forum mahasiswa se jabodet...
5591     jakarta     menelan dua kekalahan beruntun  d...
4896       kecelakaan tunggal sebuah minibus suzuki ap...
4249    jurgen klopp akhirnya memberikan klarifikasi s...
Name: content, Length: 14145, dtype: object

In [17]:
df.head(10)

,content,class
1101,", , - Ketua Harian¬†",1
2202,Beberapa hari terakhir aksi unjuk rasa atau d...,0
7349,Nama perwira tinggi dari Angkatan Udara (AU)...,1
3887,Orang yang mampu memahami emosi orang lain me...,0
1021,"- Terpidana kerusuhan 22 Mei, Rendy Bugis Pet...",0
2528,", - Dinas Pariwisata Pemerintah Kabupaten Lam...",1
4280,"IHRAM.CO.ID, JAKARTA -- Salah satu tokoh bangs...",1
6019,"-,dan Faisal Harris sempat viral beberapa wakt...",1
7468,Penyesalan selalu datang di akhir. Kalimat t...,1
1695,"- Prediksi Timnas , dalam , zona Asia dapat d...",0


In [18]:
x = df["content"]
y = df["class"]

In [19]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = .25)

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [21]:
vectorization = TfidfVectorizer()
xv_train = vectorization.fit_transform(x_train)
xv_test = vectorization.transform(x_test)

##### Logistic Regression

In [22]:
from sklearn.linear_model import LogisticRegression

In [23]:
LR = LogisticRegression()
LR.fit(xv_train, y_train)

LogisticRegression()

In [24]:
LR.score(xv_test, y_test)

0.7441334464235227

In [25]:
pred_LR = LR.predict(xv_test)

In [26]:
print(classification_report(y_test, pred_LR))

              precision    recall  f1-score   support

           0       0.72      0.62      0.67      1471
           1       0.76      0.83      0.79      2066

    accuracy                           0.74      3537
   macro avg       0.74      0.73      0.73      3537
weighted avg       0.74      0.74      0.74      3537



In [27]:
logloss = log_loss(y_test, pred_LR)
logloss

8.837443373922916

#### Decision Tree

In [28]:
from sklearn.tree import DecisionTreeClassifier

In [29]:
DT = DecisionTreeClassifier()
DT.fit(xv_train, y_train)

DecisionTreeClassifier()

In [30]:
DT.score(xv_test, y_test)

0.6875883517104892

In [31]:
pred_DT = DT.predict(xv_test)

In [32]:
print(classification_report(y_test, pred_DT))

              precision    recall  f1-score   support

           0       0.62      0.66      0.64      1471
           1       0.74      0.71      0.73      2066

    accuracy                           0.69      3537
   macro avg       0.68      0.68      0.68      3537
weighted avg       0.69      0.69      0.69      3537



In [33]:
logloss = log_loss(y_test, pred_DT)
logloss

10.79043067918389

#### Gradient Boosting Classification

In [34]:
from sklearn.ensemble import GradientBoostingClassifier

In [35]:
GBC = GradientBoostingClassifier(random_state = 0)
GBC.fit(xv_train, y_train)

GradientBoostingClassifier(random_state=0)

In [36]:
GBC.score(xv_test, y_test)

0.731128074639525

In [37]:
pred_GBC = GBC.predict(xv_test)

In [38]:
print(classification_report(y_test, pred_GBC))

              precision    recall  f1-score   support

           0       0.71      0.60      0.65      1471
           1       0.74      0.83      0.78      2066

    accuracy                           0.73      3537
   macro avg       0.73      0.71      0.72      3537
weighted avg       0.73      0.73      0.73      3537



In [39]:
logloss = log_loss(y_test, pred_GBC)
logloss

9.286641366365894

#### Random Forest Classification

In [40]:
from sklearn.ensemble import RandomForestClassifier 

In [41]:
RFC = RandomForestClassifier(random_state = 0)
RFC.fit(xv_train, y_train)

RandomForestClassifier(random_state=0)

In [42]:
RFC.score(xv_test, y_test)

0.7738196211478654

In [43]:
pred_RFC = RFC.predict(xv_test)

In [44]:
print(classification_report(y_test, pred_RFC))

              precision    recall  f1-score   support

           0       0.77      0.66      0.71      1471
           1       0.78      0.86      0.82      2066

    accuracy                           0.77      3537
   macro avg       0.77      0.76      0.76      3537
weighted avg       0.77      0.77      0.77      3537



In [45]:
logloss = log_loss(y_test, pred_RFC)
logloss

7.812107241570805

#### Manual Testing

In [46]:
def output_label(n):
    if n == 0:
        return "Clickbait"
    elif n == 1:
        return "Non-clickbait"

def manual_testing(news):
    testing_news = {"content":[news]}
    new_def_test = pd.DataFrame(testing_news)
    new_def_test["content"] = new_def_test["content"].apply(word_drop)
    new_x_test = new_def_test["content"]
    new_xv_test = vectorization.transform(new_x_test)
    pred_LR = LR.predict(new_xv_test)
    pred_DT = DT.predict(new_xv_test)
    pred_GBC = GBC.predict(new_xv_test)
    pred_RFC = RFC.predict(new_xv_test)
    
    return print("\n\nLR Prediction: {} \nDT Prediction: {} \nGBC Prediction: {} \nRFC Prediction: {}".format(output_label(pred_LR[0]),
                                                                                                              output_label(pred_DT[0]), 
                                                                                                              output_label(pred_GBC[0]), 
                                                                                                              output_label(pred_RFC[0])))

In [ ]:
news = str(input())
manual_testing(news)